In [ ]:
# default_exp core

# 00_Core

> This module contains helper functions for preparing the data

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from fastai2.tabular.all import *

We'll use the `ADULT_SAMPLE` dataset for all of our examples. This dataset's aim is to identify if an individual makes above or below $50,000

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

Next let's name our variables

In [ ]:
dep_var = 'salary'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [Categorify, FillMissing, Normalize]

And decide how we want to split and generate our `DataLoader`!

In [ ]:
splits = IndexSplitter(list(range(800,1000)))(range_of(df))
to = TabularPandas(df, procs, cat_names, cont_names, y_names="salary", splits=splits)
dls = to.dataloaders()

Now let's train an initial model to use

In [ ]:
learn = tabular_learner(dls, layers=[200,100], metrics=accuracy)
learn.fit(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.375059,0.408420,0.785000,00:08


We want to get our predictions in check for our model. We do this by passing in `SHAP` data. This has an option to pass in a test `DataFrame` or `DataLoader`. If none passed in, it will assume the validation dataset

# _prepare_data
> This function prepares some test data for how `SHAP` expects their input. You can pass in a `DataFrame` or `TabDataLoader`. If nothing is passed in, it will default to your validation data

In [ ]:
#export
def _prepare_data(learn:Learner, test_data=None):
  "Prepares train and test data for `SHAP`, pass in a learner with optional data"
  dtype = ''
  if isinstance(test_data, pd.DataFrame): dtype = 'pandas'
  elif isinstance(test_data, TabDataLoader): dtype = 'dl'
  elif test_data is None: dtype = 'train'
  else: raise ValueError('Input is not supported. Please use either a `DataFrame` or `TabularDataLoader`')
  cols = learn.dls.cat_names + learn.dls.cont_names
  X_train_cat, X_train_cont, _ = learn.dls.one_batch()
  X_train = [X_train_cat, X_train_cont]
  X_train = pd.DataFrame(np.concatenate([v.to('cpu').numpy() for v in X_train], axis=1), columns=cols)
  if dtype == 'pandas':
    dl = learn.dls.test_dl(test_data)
  elif dtype=='dl':
    dl = test_data
  else:
    dl = learn.dls[1]
    if len(dl) * learn.dls.bs > 256: 
      test_data = dl.dataset.all_cols.sample(256, replace=False)
      dl = learn.dls.test_dl(test_data)
  X_test = tensor(dl.cats).long(),tensor(dl.conts).float()
  X_test = pd.DataFrame(np.concatenate([v.to('cpu').numpy() for v in X_test], axis=1), columns=cols)
  return X_train, X_test

First, an example with a `DataFrame`

In [ ]:
X_train, X_test = _prepare_data(learn, df.iloc[:100])

In [ ]:
#hide
test_eq(len(X_train), 64)
test_eq(len(X_test), 100)

In [ ]:
X_test.head()

,workclass,education,marital-status,occupation,relationship,race,age_na,fnlwgt_na,education-num_na,age,fnlwgt,education-num
0,5.0,16.0,1.0,7.0,5.0,5.0,1.0,1.0,2.0,0.470058,-0.879315,-0.031209
1,6.0,15.0,3.0,11.0,1.0,5.0,1.0,1.0,2.0,0.543359,0.029004,-0.031209
2,5.0,9.0,1.0,0.0,2.0,5.0,1.0,1.0,2.0,-0.189644,1.770439,-0.031209
3,2.0,10.0,5.0,14.0,2.0,5.0,1.0,1.0,2.0,-0.995946,-1.324184,-0.031209
4,5.0,10.0,3.0,0.0,1.0,5.0,1.0,1.0,2.0,-0.116344,-0.238863,-0.031209


We can see that we now are working with the transformed data! Now let's do the same with a `DataLoader`

In [ ]:
dl = learn.dls.test_dl(df.iloc[:100])
X_train, X_test = _prepare_data(learn, dl)

In [ ]:
#hide
test_eq(len(X_train), 64)
test_eq(len(X_test), 100)

In [ ]:
X_test.head()

,workclass,education,marital-status,occupation,relationship,race,age_na,fnlwgt_na,education-num_na,age,fnlwgt,education-num
0,5.0,16.0,1.0,7.0,5.0,5.0,1.0,1.0,2.0,0.470058,-0.879315,-0.031209
1,6.0,15.0,3.0,11.0,1.0,5.0,1.0,1.0,2.0,0.543359,0.029004,-0.031209
2,5.0,9.0,1.0,0.0,2.0,5.0,1.0,1.0,2.0,-0.189644,1.770439,-0.031209
3,2.0,10.0,5.0,14.0,2.0,5.0,1.0,1.0,2.0,-0.995946,-1.324184,-0.031209
4,5.0,10.0,3.0,0.0,1.0,5.0,1.0,1.0,2.0,-0.116344,-0.238863,-0.031209


And now finally for absolutely nothing

In [ ]:
X_train, X_test = _prepare_data(learn)

In [ ]:
#hide
test_eq(len(X_train), 64)
test_eq(len(X_test), 200)

In [ ]:
X_test.head()

,workclass,education,marital-status,occupation,relationship,race,age_na,fnlwgt_na,education-num_na,age,fnlwgt,education-num
0,5.0,16.0,1.0,7.0,5.0,5.0,1.0,1.0,2.0,0.470058,-0.879315,-0.031209
1,6.0,15.0,3.0,11.0,1.0,5.0,1.0,1.0,2.0,0.543359,0.029004,-0.031209
2,5.0,9.0,1.0,0.0,2.0,5.0,1.0,1.0,2.0,-0.189644,1.770439,-0.031209
3,2.0,10.0,5.0,14.0,2.0,5.0,1.0,1.0,2.0,-0.995946,-1.324184,-0.031209
4,5.0,10.0,3.0,0.0,1.0,5.0,1.0,1.0,2.0,-0.116344,-0.238863,-0.031209


In [ ]:
#hide
X_train, X_test = _prepare_data(learn, 'test')

ValueError: ignored

# _predict

Now we need to grab predictions based on what shap throws back. This is a basic function you can use to get your predictions. We can't include it in the library as we need access to your current `Learn`

In [ ]:
#export
def _predict(learn:TabularLearner, data:np.array):
  "Predict function for some data on a fastai model"
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  model = learn.model.to(device)
  dl = learn.dls[0]
  nb_cat_cols = len(dl.dataset.cat_names)
  nb_cont_cols = len(dl.dataset.cont_names)
  x_cat = torch.from_numpy(data[:, :nb_cat_cols]).to(device, torch.int64)
  x_cont = torch.from_numpy(data[:, -nb_cont_cols:]).to(device, torch.float32)
  with torch.no_grad():
    pred_probs = learn.model(x_cat, x_cont).cpu().numpy()
  return pred_probs

`SHAP` will expect a numpy array for our data, so let's work with that and get some predictions!

In [ ]:
data = X_train.iloc[:5].to_numpy()
pred_probs = _predict(learn, data)

In [ ]:
#hide
test_eq(pred_probs.shape, (5,2))

Let's take a look at those predictions

In [ ]:
pred_probs.shape

(5, 2)

In [ ]:
pred_probs

array([[-0.36363047,  0.4599159 ],
       [ 0.6334366 , -0.61967963],
       [ 3.846245  , -4.456911  ],
       [ 0.49836415, -0.6335752 ],
       [-0.27864254,  0.34394863]], dtype=float32)

And now we can do whatever we need to!